In [ ]:
# ============================================================================
# HÜCRE -1: KÜTÜPHANE GÜNCELLEME (HER ŞEYDEN ÖNCE BUNU ÇALIŞTIR)
# ============================================================================
print("Tüm gerekli kütüphaneler en güncel versiyonlara yükseltiliyor...")
print("Bu işlem birkaç dakika sürebilir...")

# '!' işareti, bu komutun bir Python kodu değil, bir terminal komutu olduğunu belirtir.
!pip install --upgrade --quiet google-generativeai langchain-google-genai langchain langchain-chroma langchain-huggingface

print("\n✅ Güncelleme tamamlandı! LÜTFEN BİR SONRAKİ ADIMA GEÇİN.")

In [ ]:
# ============================================================================
# HÜCRE 0: MODEL TEŞHİS ARACI
# Bu kod, API anahtarınızla hangi Gemini modellerini kullanabileceğinizi listeler.
# ============================================================================
import google.generativeai as genai
import os

print("API Anahtarınız için kullanılabilir Gemini modelleri listeleniyor...")
print("="*60)

try:
    api_key = os.environ.get('GOOGLE_API_KEY')
    if not api_key:
        raise ValueError("API Anahtarı bulunamadı! Lütfen 'GOOGLE_API_KEY' ortam değişkenini kontrol edin.")
    
    genai.configure(api_key=api_key)

    # API'den, 'generateContent' metodunu destekleyen tüm modelleri listelemesini isteyelim.
    model_found = False
    for m in genai.list_models():
        if 'generateContent' in m.supported_generation_methods:
            print(f"✅ Model Adı: {m.name}")
            print(f"   Açıklama: {m.description}\n")
            model_found = True
            
    if not model_found:
        print("❌ 'generateContent' metodunu destekleyen hiçbir model bulunamadı.")
        print("   Lütfen Google AI Studio veya Google Cloud Console üzerinden projenizi ve API anahtarınızı kontrol edin.")

except Exception as e:
    print(f"🚨 Modeller listelenirken bir hata oluştu: {e}")

print("="*60)
print("Teşhis tamamlandı. Lütfen yukarıdaki 'Model Adı' listesinden birini kopyalayın.")

In [ ]:
# ======================================================================================
# BÖLÜM 1: PROJE GÜNLÜĞÜ - Veri Üretme Serüveni ve Karşılaşılan Zorluklar
# AMAÇ: Bu blok, veri zenginleştirme sürecinde karşılaşılan hataları,
# denenen çözüm yollarını ve nihai çözüme nasıl ulaşıldığını belgelemektedir.
# Bu sayede proje sunumunda, problem çözme yeteneği gösterilebilir.
#
# Bu bölümdeki kodlar çalıştırılamaz, sadece belgelendirme amaçlıdır.
# ======================================================================================

"""
########################################################################################
# DENEME 1: Google Gemini API ile Veri Üretme (İlk Hata)
########################################################################################
# HEDEF: Google Gemini API'sinin ücretsiz katmanını kullanarak cevap varyasyonları üretmek.
#
# KARŞILAŞILAN SORUN: Günlük Kota Hatası (429 ResourceExhausted)
#   - Hata Mesajı: "Quota exceeded for metric: ...generate_content_free_tier_requests, limit: 200"
#   - TEŞHİS: Script bir süre (yaklaşık 90-100 soru) çalıştıktan sonra duruyordu.
#     Bunun sebebinin, Google'ın ücretsiz API katmanına koyduğu "Günlük Toplam İstek Limiti"
#     olduğu anlaşıldı. Dakikalarca beklemek bu sorunu çözmüyordu çünkü limit günlük olarak sıfırlanıyordu.
#
#   - ÇÖZÜM PLANI: Bu işlemi tamamlamak için ya günlere bölerek manuel çalıştıracaktık ya da
#     farklı bir API sağlayıcısına geçecektik. Daha esnek bir çözüm arayışına girdik.
########################################################################################


########################################################################################
# DENEME 2: Hugging Face API'ye Geçiş (Kütüphane ve Model Hataları)
########################################################################################
# HEDEF: Google'ın günlük limitini aşmak için Hugging Face'in ücretsiz Inference API'ını kullanmak.
#        Model olarak Meta'nın güçlü Llama-3 modeli seçildi.
#
# KARŞILAŞILAN SORUN 1: Kütüphane Uyumluluk Hatası ('thinking' Hatası)
#   - Hata Mesajı: "Model.__init__() got an unexpected keyword argument 'thinking'"
#   - TEŞHİS: Bu hatanın temel sebebi, `google-generativeai` ve `langchain` gibi
#     kütüphanelerin bilgisayarımızda yüklü olan versiyonlarının eski olmasıydı.
#     API'dan gelen yeni bir alanı tanıyamıyorlardı.
#   - ÇÖZÜM: Projenin en başına bir kurulum hücresi eklenerek tüm kütüphaneler
#     `!pip install --upgrade` komutuyla en güncel sürümlerine yükseltildi.
#
# KARŞILAŞILAN SORUN 2: Model Görev Uyumsuzluğu Hatası
#   - Hata Mesajı: "Model ... is not supported for task text-generation... Supported task: conversational"
#   - TEŞHİS: LangChain'deki `HuggingFaceEndpoint` aracının, Llama-3-Instruct gibi
#     modelleri "sohbet" formatı yerine "düz metin üretme" formatında çağırmaya çalıştığı anlaşıldı.
#
# KARŞILAŞILAN SORUN 3: Yanlış Sarmalayıcı (Wrapper) Kullanımı
#   - Hata Mesajı: "ValidationError: ... llm Field required"
#   - TEŞHİS: `ChatHuggingFace` aracının, doğrudan model bilgileriyle değil, önce
#     oluşturulmuş bir `HuggingFaceEndpoint` nesnesini `llm=` parametresi olarak beklediği
#     tespit edildi. Kod bu yapıya uygun değildi.
#
#   - ÇÖZÜM PLANI: Bu üç sorunu çözmek için kod, temel bağlantıyı `HuggingFaceEndpoint` ile
#     kuracak ve ardından bu bağlantıyı `ChatHuggingFace` sarmalayıcısına verecek şekilde
#     yeniden düzenlendi. Bu, hem doğru görev formatını (`conversational`) kullanmamızı sağladı
#     hem de kütüphanenin beklediği doğru kurulumu gerçekleştirdi.
########################################################################################


########################################################################################
# DENEME 3: Hugging Face ile Uzun Süreli Çalışma (Aylık Kredi Hatası)
########################################################################################
# HEDEF: Düzeltilmiş Hugging Face koduyla tüm veri setini işlemek.
#
# KARŞILAŞILAN SORUN: Ödeme Gerekli Hatası (402 Payment Required)
#   - Hata Mesajı: "You have exceeded your monthly included credits for Inference Providers."
#   - TEŞHİS: Script'in bir süre sorunsuz çalıştıktan sonra durması, bu sefer de
#     Hugging Face'in "aylık ücretsiz kullanım kredisinin" bittiğini gösterdi.
#     Bu, günlük sıfırlanan bir limit değildi ve bu yolu da tıkamış oldu.
#     Bu durum, ücretsiz katmanların büyük ölçekli toplu işlemler için uygun
#     olmadığını, daha çok deneme ve düşük trafikli uygulamalar için tasarlandığını kanıtladı.
########################################################################################

"""
# ======================================================================================
# HÜCRE 1: SENTETİK VERİ ÜRETİCİ (Hugging Face - Tam Açıklamalı ve Güvenli Versiyon)
# AMAÇ: API limitlerine takılmadan, uzun süre boyunca hatasız bir şekilde çalışarak
# veri setini zenginleştirmek.
# YÖNTEM: Hugging Face Inference API'ı ve Llama-3 modeli kullanılır.
# ÖZELLİKLER:
#   - Kaldığı Yerden Devam Etme: Script durdurulursa veya hata alırsa, yeniden
#     çalıştığında kaldığı yerden devam eder, ilerleme kaybolmaz.
#   - Hata Yönetimi: Tek bir API çağrısı hata verse bile tüm işlem çökmez.
#   - Hız Limiti Koruması: API limitlerine takılmamak için istekler arasında bekler.
# ======================================================================================

# -- Gerekli Kütüphanelerin İçe Aktarılması --
import os  # Ortam değişkenlerini (API anahtarı gibi) okumak için
import json  # Veri setimiz JSON formatında olduğu için verileri okumak ve yazmak için
import time  # API istekleri arasında bekleme yapmak için

# LangChain'in Hugging Face ile iletişim kurmamızı sağlayan araçları
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
# LLM'e göndereceğimiz talimatları (prompt) formatlamak için
from langchain.prompts import ChatPromptTemplate
# LLM'den gelen cevabı temiz bir metne dönüştürmek için
from langchain.schema.output_parser import StrOutputParser

print("🤖 Sentetik Cevap Üretici (Hugging Face - Güvenli Mod) Başlatılıyor...")

# --- AYARLAR ---
# Bu bölümde, script'in çalışması için gerekli olan dosya yolları ve parametreler tanımlanır.

# Orijinal, temizlenmiş veri setimizin tam dosya yolu.
INPUT_FILE_PATH = r"C:\Users\yedis\Desktop\yehu\Github Repo\MentorMate-SSS\output\enriched_dataset.jsonl"
# Üretilecek yeni cevapların kaydedileceği dosyanın tam yolu.
OUTPUT_FILE_PATH = r"C:\Users\yedis\Desktop\yehu\Github Repo\MentorMate-SSS\output\generated_data_hf.jsonl"
# Orijinal veri setindeki her bir soru için kaç adet yeni cevap varyasyonu üretileceği.
VARIANTS_PER_QUESTION = 2 

# GÜVENLİK AYARI: Her API isteği arasında beklenecek saniye.
# Bu, Hugging Face'in ücretsiz katmanının hız limitlerine takılma riskini en aza indirir.
DELAY_SECONDS = 25 

# Hugging Face üzerinde kullanmak istediğimiz açık kaynak modelin kimliği.
HF_MODEL_NAME = "meta-llama/Meta-Llama-3-8B-Instruct" 

# --- API ve Model Yapılandırması ---
# Bilgisayarın ortam değişkenlerinden 'HUGGINGFACE_HUB_TOKEN' adlı anahtarı okuyoruz.
hf_token = os.environ.get('HUGGINGFACE_HUB_TOKEN')
if not hf_token:
    # Eğer anahtar bulunamazsa, kullanıcıyı uyararak programı durduruyoruz.
    raise ValueError("Hugging Face API Token bulunamadı! Lütfen 'HUGGINGFACE_HUB_TOKEN' ortam değişkenini ayarlayın.")

# 1. Aşama: Hugging Face'teki model sunucusuna temel bir bağlantı (endpoint) kuruyoruz.
# Bu bağlantı, modelin nasıl davranacağını belirleyen parametreleri içerir.
endpoint_llm = HuggingFaceEndpoint(
    repo_id=HF_MODEL_NAME,  # Hangi modeli kullanacağımız
    huggingfacehub_api_token=hf_token,  # Kimlik doğrulama için API token'ımız
    temperature=0.7,  # Modelin ne kadar "yaratıcı" olacağı (0'a yakın daha net, 1'e yakın daha rastgele)
    max_new_tokens=256,  # Modelin üreteceği cevabın maksimum uzunluğu (kelime/token sayısı)
    repetition_penalty=1.2,  # Modelin aynı kelimeleri tekrar etmesini engellemeye yönelik bir ceza puanı
)

# 2. Aşama: Bu temel bağlantıyı, LangChain'in sohbet formatında iletişim kurabilen
# `ChatHuggingFace` sarmalayıcısına (wrapper) veriyoruz. Bu, modelle sohbet eder gibi konuşmamızı sağlar.
llm = ChatHuggingFace(llm=endpoint_llm)
print(f"✅ Hugging Face Endpoint ve '{HF_MODEL_NAME}' modeli yapılandırıldı.")

# --- Prompt ve Zincir Yapılandırması ---
# LLM'e ne yapması gerektiğini anlatan detaylı talimat metnimiz.
# Llama-3'ün kendi özel formatına (<|...|>) tam uyan bir yapı kullanıyoruz.
prompt_template_str = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
Sen bir metin yeniden yazma asistanısın. Görevin, verilen bir cevabı, anlamını tamamen koruyarak ama farklı kelimelerle yeniden yazmaktır. Sadece ve sadece yeniden yazılmış cevabı döndür. Başka hiçbir açıklama ekleme.<|eot_id|><|start_header_id|>user<|end_header_id|>
Orijinal Soru: {soru}
Orijinal Cevap: {orijinal_cevap}
Yeniden Yazılmış Cevap:<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""
# Yukarıdaki metni LangChain'in kullanabileceği bir şablon nesnesine dönüştürüyoruz.
prompt = ChatPromptTemplate.from_template(prompt_template_str)
# LLM'den gelen cevabı, içindeki ekstra bilgileri temizleyerek saf bir metne dönüştüren bir ayrıştırıcı.
output_parser = StrOutputParser()

# LangChain Expression Language (LCEL) kullanarak adımları birbirine zincirliyoruz.
# Okunuşu: Prompt'u al -> LLM'e gönder -> Gelen cevabı ayrıştırıcıdan geçir.
chain = prompt | llm | output_parser
print("✅ LangChain zinciri oluşturuldu.")

# --- Veri Üretme Döngüsü ---
# Tüm bu uzun işlemi, herhangi bir kritik hata durumunda çökmemesi için bir try-except bloğu içine alıyoruz.
try:
    # --- Kaldığı Yeri Hesaplama ---
    start_index = 0
    # Eğer daha önce bir çıktı dosyası oluşturulmuşsa...
    if os.path.exists(OUTPUT_FILE_PATH):
        # ...bu dosyayı aç ve içindeki satır sayısını oku.
        with open(OUTPUT_FILE_PATH, 'r', encoding='utf-8') as f:
            lines_in_output = len(f.readlines())
        # Üretilen varyasyon sayısından yola çıkarak, orijinal dosyada hangi sorudan devam etmemiz gerektiğini hesapla.
        start_index = lines_in_output // VARIANTS_PER_QUESTION
        print(f"\n Mevcut dosyada {lines_in_output} varyasyon bulundu.")
        if start_index > 0:
            print(f" İşleme {start_index + 1}. sorudan devam edilecek.")

    # Girdi dosyasını okuma ('r') modunda, çıktı dosyasını ise 'append' ('a' - sonuna ekle) modunda aç.
    # 'a' modu, var olan veriyi silmeden yeni verileri dosyanın sonuna eklememizi sağlar.
    with open(INPUT_FILE_PATH, 'r', encoding='utf-8') as infile, \
         open(OUTPUT_FILE_PATH, 'a', encoding='utf-8') as outfile:
        
        # Orijinal dosyadaki tüm satırları oku
        lines = infile.readlines()
        # Sadece kaldığımız yerden itibaren olan soruları işleme al
        lines_to_process = lines[start_index:]
        
        total_lines = len(lines)
        print(f" Orijinal dosyada toplam {total_lines} soru var. Kalan {len(lines_to_process)} soru işlenecek.")

        if not lines_to_process:
            print("Tüm sorular zaten işlenmiş. İşlem tamamlandı.")
        else:
            print(f"Her API çağrısı arasında {DELAY_SECONDS} saniye beklenecektir...")

        # İşlenecek her bir satır (soru-cevap çifti) için döngüyü başlat
        for i, line in enumerate(lines_to_process, start=start_index):
            original_data = json.loads(line)
            question = original_data.get("question")
            answer = original_data.get("answer")
            if not question or not answer: continue

            print(f"\n[{i+1}/{total_lines}] İşleniyor: '{question[:50]}...'")
            # Her soru için belirlediğimiz sayıda varyasyon üret
            for j in range(VARIANTS_PER_QUESTION):
                print(f"  -> Varyasyon {j+1}/{VARIANTS_PER_QUESTION} üretiliyor...")
                # Bu iç try-except bloğu, tek bir API çağrısı hata verse bile döngünün devam etmesini sağlar.
                try:
                    # Zinciri çağırarak LLM'den yeni cevabı istiyoruz.
                    new_answer = chain.invoke({
                        "soru": question,
                        "orijinal_cevap": answer
                    })
                    
                    # Yeni oluşturulan veriyi formatlayıp dosyaya yazıyoruz.
                    new_data_entry = {"question": question, "answer": new_answer.strip()}
                    outfile.write(json.dumps(new_data_entry, ensure_ascii=False) + "\n")

                except Exception as e:
                    print(f"    !! API Hatası: {e}. Bu varyasyon atlanıyor.")
                    # Eğer hata bir kota hatası ise, işlemi tamamen durdur.
                    if "quota" in str(e).lower() or "payment required" in str(e).lower():
                        print("    !! KOTA LİMİTİNE ULAŞILDI. İşlem durduruluyor. Lütfen daha sonra tekrar deneyin.")
                        raise e # Script'i durdurmak için hatayı yeniden fırlat.
                
                # Hız limitine takılmamak için bekle.
                time.sleep(DELAY_SECONDS)

    print("\n" + "="*80 + f"\n✅ İŞLEM TAMAMLANDI! Üretilen veriler '{OUTPUT_FILE_PATH}' dosyasına kaydedildi.\n" + "="*80)

except Exception as e:
    # Üstteki 'raise e' komutu çalıştığında veya dosyalarla ilgili bir sorun olduğunda bu mesaj gösterilir.
    print(f"\nİşlem durduruldu. Üretilen veriler kaydedildi. Limitin sıfırlanmasını bekledikten sonra aynı hücreyi tekrar çalıştırarak devam edebilirsiniz.")

In [ ]:
# ============================================================================
# HÜCRE: VERİ DOKTORU - TEKRAR EDEN VE POTANSİYEL YANLIŞ CEVAPLARI TESPİT ETME
# ============================================================================
import json
from collections import defaultdict

# Lütfen veri setinin yolunun doğru olduğundan emin ol
INPUT_FILE_PATH = r"C:\Users\yedis\Desktop\yehu\Github Repo\MentorMate-SSS\output\enriched_dataset.jsonl"
REPORT_FILE_PATH = r"C:\Users\yedis\Desktop\yehu\Github Repo\MentorMate-SSS\output\data_quality_report.txt"

# Cevapları anahtar, soruları ise bir liste olarak tutacak bir sözlük oluşturalım
answers_to_questions = defaultdict(list)

print(f"'{INPUT_FILE_PATH}' dosyası analiz ediliyor...")

# Dosyayı oku ve cevaplara göre soruları grupla
try:
    with open(INPUT_FILE_PATH, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                data = json.loads(line)
                if 'question' in data and 'answer' in data:
                    answers_to_questions[data['answer']].append(data['question'])
            except json.JSONDecodeError:
                print(f"Uyarı: Bir satır okunamadı, geçiliyor.")
                continue

    print("Analiz tamamlandı. Rapor oluşturuluyor...")

    # Rapor dosyasını yaz
    with open(REPORT_FILE_PATH, 'w', encoding='utf-8') as f:
        # En çok tekrar eden cevapları başa alalım
        sorted_answers = sorted(answers_to_questions.items(), key=lambda item: len(item[1]), reverse=True)
        
        for answer, questions in sorted_answers:
            f.write("="*80 + "\n")
            f.write(f"CEVAP (Toplam {len(questions)} soru için kullanılmış):\n")
            f.write(f"{answer}\n")
            f.write("-" * 20 + "\n")
            f.write("Bu Cevabın Atandığı Sorular:\n")
            for i, question in enumerate(questions, 1):
                f.write(f"  {i}. {question}\n")
            f.write("\n")

    print(f"✅ Rapor başarıyla '{REPORT_FILE_PATH}' dosyasına kaydedildi.")
    print("Lütfen bu dosyayı açıp mantıksız Soru-Cevap eşleşmelerini kontrol et.")

except FileNotFoundError:
    print(f"HATA: Dosya bulunamadı: {INPUT_FILE_PATH}")
except Exception as e:
    print(f"Beklenmedik bir hata oluştu: {e}")

In [3]:
# ======================================================================================
# HÜCRE 1.4: VERİ ÜRETİCİ (Google Gemini 2.0 - Güvenli Devam Etme Modu)
# AMAÇ: Google'ın günlük limitine takılsa bile, ertesi gün kaldığı yerden
# devam edebilen bir veri üretme script'i.
# ======================================================================================
import os
import json
import time

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

print("🤖 Sentetik Cevap Üretici (Google Gemini Versiyonu) Başlatılıyor...")

# --- AYARLAR ---
INPUT_FILE_PATH = r"C:\Users\yedis\Desktop\yehu\Github Repo\MentorMate-SSS\output\enriched_dataset.jsonl"
# Hugging Face'ten ayrı bir dosya oluşturalım ki karışmasın
OUTPUT_FILE_PATH = r"C:\Users\yedis\Desktop\yehu\Github Repo\MentorMate-SSS\output\generated_data_google.jsonl"
VARIANTS_PER_QUESTION = 2 
# Google'ın dakikalık limiti (RPM) daha yüksek olduğu için bekleme süresini daha düşük tutabiliriz.
DELAY_SECONDS = 15 
LLM_MODEL_NAME = "models/gemini-2.0-flash" 

# --- API ve Model Yapılandırması ---
api_key = os.environ.get('GOOGLE_API_KEY-cosmo')
if not api_key:
    raise ValueError("Google API Anahtarı bulunamadı! Lütfen 'GOOGLE_API_KEY' ortam değişkenini ayarlayın.")

llm = ChatGoogleGenerativeAI(model=LLM_MODEL_NAME, temperature=0.7, google_api_key=api_key)
print(f"✅ LangChain ve Gemini modeli ({LLM_MODEL_NAME}) veri üretimi için yapılandırıldı.")

# --- Prompt ve Zincir ---
prompt_template_str = """
Sen bir metin yeniden yazma asistanısın. Görevin, verilen bir cevabı, anlamını tamamen koruyarak ama farklı kelimelerle yeniden yazmaktır. Sadece ve sadece yeniden yazılmış cevabı döndür. Başka hiçbir açıklama ekleme.

Orijinal Soru: {soru}
Orijinal Cevap: {orijinal_cevap}
Yeniden Yazılmış Cevap:
"""
prompt = ChatPromptTemplate.from_template(prompt_template_str)
output_parser = StrOutputParser()
chain = prompt | llm | output_parser
print("✅ LangChain zinciri oluşturuldu.")

# --- Kaldığı Yeri Hesaplama ---
start_index = 0
if os.path.exists(OUTPUT_FILE_PATH):
    with open(OUTPUT_FILE_PATH, 'r', encoding='utf-8') as f:
        lines_in_output = len(f.readlines())
        start_index = lines_in_output // VARIANTS_PER_QUESTION
        print(f"\n Mevcut dosyada {lines_in_output} varyasyon bulundu.")
        if start_index > 0:
            print(f" İşleme {start_index + 1}. sorudan devam edilecek.")

# --- Veri Üretme Döngüsü ---
try:
    # DOSYA MODU 'a' (append/ekleme) olarak ayarlandı!
    with open(INPUT_FILE_PATH, 'r', encoding='utf-8') as infile, \
         open(OUTPUT_FILE_PATH, 'a', encoding='utf-8') as outfile:
        
        lines = infile.readlines()
        lines_to_process = lines[start_index:]
        total_lines = len(lines)
        
        print(f" Orijinal dosyada toplam {total_lines} soru var. Kalan {len(lines_to_process)} soru işlenecek.")
        if not lines_to_process:
            print("Tüm sorular zaten işlenmiş. İşlem tamamlandı.")
        else:
            print("İşlem başlıyor...")

        for i, line in enumerate(lines_to_process, start=start_index):
            original_data = json.loads(line)
            question = original_data.get("question")
            answer = original_data.get("answer")
            if not question or not answer: continue

            print(f"\n[{i+1}/{total_lines}] İşleniyor: '{question[:50]}...'")
            for j in range(VARIANTS_PER_QUESTION):
                print(f"  -> Varyasyon {j+1}/{VARIANTS_PER_QUESTION} üretiliyor...")
                try:
                    new_answer = chain.invoke({"soru": question, "orijinal_cevap": answer})
                    new_data_entry = {"question": question, "answer": new_answer.strip()}
                    outfile.write(json.dumps(new_data_entry, ensure_ascii=False) + "\n")
                except Exception as e:
                    print(f"    !! API Hatası: {e}. Bu varyasyon atlanıyor.")
                    # Günlük limite takıldığımızda script'in durması için hatayı yeniden fırlatabiliriz.
                    if "quota" in str(e).lower():
                        print("    !! GÜNLÜK KOTA LİMİTİNE ULAŞILDI. Lütfen yarın tekrar deneyin.")
                        raise e # Script'i durdur.
                
                time.sleep(DELAY_SECONDS)

    print("\n" + "="*80 + f"\n✅ İŞLEM TAMAMLANDI! Üretilen veriler '{OUTPUT_FILE_PATH}' dosyasına kaydedildi.\n" + "="*80)

except Exception as e:
    print(f"İşlem durduruldu: {e}")

🤖 Sentetik Cevap Üretici (Google Gemini Versiyonu) Başlatılıyor...
✅ LangChain ve Gemini modeli (models/gemini-2.0-flash) veri üretimi için yapılandırıldı.
✅ LangChain zinciri oluşturuldu.

 Mevcut dosyada 2047 varyasyon bulundu.
 İşleme 1024. sorudan devam edilecek.
 Orijinal dosyada toplam 1077 soru var. Kalan 54 soru işlenecek.
İşlem başlıyor...

[1024/1077] İşleniyor: 'Yetenek havuzuna dahil edilme, bootcamp'in bir çık...'
  -> Varyasyon 1/2 üretiliyor...
  -> Varyasyon 2/2 üretiliyor...

[1025/1077] İşleniyor: 'Yetenek havuzuna dahil edilme, bootcamp'in bir çık...'
  -> Varyasyon 1/2 üretiliyor...
  -> Varyasyon 2/2 üretiliyor...

[1026/1077] İşleniyor: 'Sponsor şirket, başarıyla tamamlayanları kendi yet...'
  -> Varyasyon 1/2 üretiliyor...
  -> Varyasyon 2/2 üretiliyor...

[1027/1077] İşleniyor: 'Sponsor şirket, başarıyla tamamlayanları kendi yet...'
  -> Varyasyon 1/2 üretiliyor...
  -> Varyasyon 2/2 üretiliyor...

[1028/1077] İşleniyor: 'Başarılı bootcamp katılımcıları sponsor 